In [44]:
import numpy as np
import librosa
import os
import wave

def preprocess_audio(audio_path):
    # Carregar o arquivo de áudio
    with wave.open(audio_path, 'rb') as audio_file:
        # Obter os parâmetros do áudio
        sr = audio_file.getframerate()
        n_channels = audio_file.getnchannels()
        sample_width = audio_file.getsampwidth()
        n_frames = audio_file.getnframes()

        # Ler todos os quadros de áudio como bytes
        audio_frames = audio_file.readframes(n_frames)
    print('audio_frames carregado')
    # Converter os bytes em um array numpy com o formato correto
    audio_array = np.frombuffer(audio_frames, dtype=np.int16)
    print('audio_frames carregado 2')

    # Redimensionar o array para um vetor unidimensional
    if n_channels > 1:
        audio_array = audio_array.reshape(-1, n_channels)[:, 0]
    print('audio_frames carregado 3')

    # Padronizar a taxa de amostragem em 22 kHz
    y = audio_array.astype(np.float32) / np.iinfo(sample_width).max
    y = np.interp(np.arange(0, len(y), len(y) / (sr * 22)), np.arange(0, len(y)), y)
    
    print('audio_frames carregado 4')

    # Extrair as características do áudio
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=22050)
    rmse = librosa.feature.rms(y=y)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=22050)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=22050)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=22050)
    zcr = librosa.feature.zero_crossing_rate(y)

    print('audio_frames carregado 5')

    # Concatenar as características em um vetor unidimensional
    features = np.concatenate([chroma_stft.mean(axis=1), rmse, spec_cent, spec_bw, rolloff, zcr])
    print('audio_frames carregado 6')
    # Redimensionar o vetor de características para o formato esperado pela rede neural
    features = features.reshape(1, -1)
    print('audio_frames carregado 7')
    return features


In [29]:
from keras.models import load_model

def load_model_from_h5(h5_file_path):
    model = load_model(h5_file_path)
    return model

In [30]:
import os

def get_h5_files(folder_path):
    h5_files = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.h5'):
            h5_files.append(os.path.join(folder_path, filename))
    return h5_files

In [31]:
import h5py

def test_audio_file(h5_folder_path):
    # Carregar o arquivo de áudio e extrair as características
    features = preprocess_audio()

    # Percorrer cada arquivo h5 na pasta
    for h5_file_path in get_h5_files(h5_folder_path):
        # Carregar o modelo a partir do arquivo h5
        model = load_model(h5_file_path)

        # Realizar a predição para o arquivo de áudio
        prediction = model.predict(features)

        # Imprimir o resultado da predição
        print(f'{os.path.basename(h5_file_path)}: {prediction[0][0]}')

In [32]:
import os
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report

def evaluate_models(audio_data):
    # lista os arquivos .h5
    h5_files = [f for f in os.listdir('.') if f.endswith('.h5')]

    # carrega cada modelo e avalia o desempenho no conjunto de teste
    metrics = {}
    for h5_file in h5_files:
        # carrega o modelo
        model = load_model(h5_file)

        # avalia o desempenho no conjunto de teste
        y_true = [0] # ou [1], dependendo se o áudio é de um paciente com parkinson ou não
        y_pred = model.predict(audio_data)
        y_pred = [1 if p > 0.5 else 0 for p in y_pred] # converte as probabilidades em classes (0 ou 1)
        metrics[h5_file] = classification_report(y_true, y_pred)

    return metrics


In [45]:
audio_path = 'audio/paciente1.wav'
features = preprocess_audio(audio_path)


audio_frames carregado
audio_frames carregado 2
audio_frames carregado 3
audio_frames carregado 4


KeyboardInterrupt: 